In [1]:
import numpy as np
import cv2
import torch
from torchvision import transforms
from utils.datasets import letterbox
from utils.general import non_max_suppression_kpt
from utils.plots import output_to_keypoint, plot_skeleton_kpts
from models.yolo import Model

# Add the custom class to the safe globals list
torch.serialization.add_safe_globals([Model])

# Initialize device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load YOLOv7-pose model
weights = torch.load('yolov7-w6-pose.pt', map_location=device, weights_only=False)
model = weights['model']
_ = model.float().eval()

if torch.cuda.is_available():
    model.half().to(device)

def calculate_length_factor(shoulder, torso):
    """
    Calculate the length factor using Euclidean distance.
    :param shoulder: Coordinates of the shoulder (x, y).
    :param torso: Coordinates of the torso (x, y).
    :return: Length factor.
    """
    return np.sqrt((shoulder[0] - torso[0])**2 + (shoulder[1] - torso[1])**2)

def detect_fall(keypoints, threshold=0.5, alpha=0.5):
    """
    Detect fall based on keypoints.
    :param keypoints: Array of keypoints (17 keypoints, each with x, y, confidence).
    :param threshold: Confidence threshold for keypoints.
    :param alpha: Adjustment factor for fall detection.
    :return: True if fall is detected, False otherwise.
    """
    # Indices for keypoints (COCO format)
    LEFT_SHOULDER = 5
    RIGHT_SHOULDER = 6
    LEFT_HIP = 11
    RIGHT_HIP = 12
    LEFT_ANKLE = 13
    RIGHT_ANKLE = 14

    # Get keypoints and confidence scores
    left_shoulder = keypoints[LEFT_SHOULDER * 3: (LEFT_SHOULDER + 1) * 3]
    right_shoulder = keypoints[RIGHT_SHOULDER * 3: (RIGHT_SHOULDER + 1) * 3]
    left_hip = keypoints[LEFT_HIP * 3: (LEFT_HIP + 1) * 3]
    right_hip = keypoints[RIGHT_HIP * 3: (RIGHT_HIP + 1) * 3]
    left_ankle = keypoints[LEFT_ANKLE * 3: (LEFT_ANKLE + 1) * 3]
    right_ankle = keypoints[RIGHT_ANKLE * 3: (RIGHT_ANKLE + 1) * 3]

    # Check confidence scores
    if (left_shoulder[2] < threshold or right_shoulder[2] < threshold or
        left_hip[2] < threshold or right_hip[2] < threshold or
        left_ankle[2] < threshold or right_ankle[2] < threshold):
        return False  # Skip if any keypoint is not confident

    # Calculate length factor
    length_factor = calculate_length_factor(left_shoulder[:2], left_hip[:2])

    # Check if shoulders are below ankles (fall condition)
    if (left_shoulder[1] <= left_ankle[1] + alpha * length_factor and
        right_shoulder[1] <= right_ankle[1] + alpha * length_factor):
        # Calculate body dimensions
        body_height = abs(left_shoulder[1] - left_ankle[1])
        body_width = abs(left_shoulder[0] - right_shoulder[0])

        # Check if body height is less than body width
        if body_height < body_width:
            return True

    return False

# Real-time fall detection
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess the frame
    image = letterbox(frame, 960, stride=64, auto=True)[0]
    image_ = image.copy()
    image = transforms.ToTensor()(image)
    image = torch.tensor(np.array([image.numpy()]))

    if torch.cuda.is_available():
        image = image.half().to(device)

    # Run inference
    with torch.no_grad():
        output, _ = model(image)
        output = non_max_suppression_kpt(output, 0.25, 0.65, nc=model.yaml['nc'], nkpt=model.yaml['nkpt'], kpt_label=True)
        output = output_to_keypoint(output)

    nimg = image[0].permute(1, 2, 0) * 255
    nimg = nimg.cpu().numpy().astype(np.uint8)
    nimg = cv2.cvtColor(nimg, cv2.COLOR_RGB2BGR)

    for idx in range(output.shape[0]):
        keypoints = output[idx, 7:].T
        plot_skeleton_kpts(nimg, keypoints, 3)

        # Detect fall
        if detect_fall(keypoints):
            cv2.putText(nimg, "Fall Detected!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Display the result
    cv2.imshow("Fall Detection", nimg)

    # Break if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # Clear GPU cache
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

cap.release()
cv2.destroyAllWindows()

C:\Users\LENOVO\anaconda3\envs\yolov7_env\lib\site-packages\torch\functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\TensorShape.cpp:3638.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [4]:
import numpy as np
import cv2
import torch
from torchvision import transforms
from utils.datasets import letterbox
from utils.general import non_max_suppression_kpt
from utils.plots import output_to_keypoint, plot_skeleton_kpts
from models.yolo import Model

# Add the custom class to the safe globals list
torch.serialization.add_safe_globals([Model])

# Initialize device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load YOLOv7-pose model
weights = torch.load('yolov7-w6-pose.pt', map_location=device, weights_only=False)
model = weights['model']
_ = model.float().eval()

if torch.cuda.is_available():
    model.half().to(device)

def calculate_length_factor(shoulder, torso):
    return np.sqrt((shoulder[0] - torso[0])**2 + (shoulder[1] - torso[1])**2)

def detect_fall(keypoints, threshold=0.5, alpha=0.5):
    LEFT_SHOULDER, RIGHT_SHOULDER, LEFT_HIP, RIGHT_HIP, LEFT_ANKLE, RIGHT_ANKLE = 5, 6, 11, 12, 13, 14
    
    keypoints_conf = [keypoints[i * 3: (i + 1) * 3] for i in [LEFT_SHOULDER, RIGHT_SHOULDER, LEFT_HIP, RIGHT_HIP, LEFT_ANKLE, RIGHT_ANKLE]]
    if any(kp[2] < threshold for kp in keypoints_conf):
        return False
    
    length_factor = calculate_length_factor(keypoints_conf[0][:2], keypoints_conf[2][:2])
    
    if (keypoints_conf[0][1] <= keypoints_conf[4][1] + alpha * length_factor and
        keypoints_conf[1][1] <= keypoints_conf[5][1] + alpha * length_factor):
        
        body_height = abs(keypoints_conf[0][1] - keypoints_conf[4][1])
        body_width = abs(keypoints_conf[0][0] - keypoints_conf[1][0])
        
        if body_height < body_width:
            return True
    
    return False

# Path to video file
video_path = "C:\\Users\\LENOVO\\Documents\\A Skripsi\\datasets\\FallDataset\\Dataset\\Coffee_room_01\\Videos\\video (1).avi"
cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    image = letterbox(frame, 640, stride=64, auto=True)[0]
    image_ = image.copy()
    image = transforms.ToTensor()(image)
    image = torch.tensor(np.array([image.numpy()]))

    if torch.cuda.is_available():
        image = image.half().to(device)

    with torch.no_grad():
        output, _ = model(image)
        output = non_max_suppression_kpt(output, 0.25, 0.65, nc=model.yaml['nc'], nkpt=model.yaml['nkpt'], kpt_label=True)
        output = output_to_keypoint(output)

    nimg = image[0].permute(1, 2, 0) * 255
    nimg = nimg.cpu().numpy().astype(np.uint8)
    nimg = cv2.cvtColor(nimg, cv2.COLOR_RGB2BGR)

    for idx in range(output.shape[0]):
        keypoints = output[idx, 7:].T
        plot_skeleton_kpts(nimg, keypoints, 3)

        if detect_fall(keypoints):
            cv2.putText(nimg, "Fall Detected!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow("Fall Detection", nimg)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    if torch.cuda.is_available():
        torch.cuda.empty_cache()

cap.release()
cv2.destroyAllWindows()
